In [17]:
import camelot
import pandas as pd
import numpy as np
import cred as cred
import os

In [18]:
filenames = os.listdir('./pdf')[1:]
filenames[3]

'Credit Card Statement 21-10.pdf'

In [19]:
def df_creator(temp):
    
    print('creating df.....')
    
    table = pd.DataFrame(temp)

    table['amt'] = table['amt'].str.replace(" ","")
    table['cb'] = table['cb'].str.replace(" ","")

    table['amt_value'] = table['amt'].str.slice(stop=-2)
    table['amt_type'] = table['amt'].str.slice(start=-2)
    table['cb_value'] = table['cb'].str.slice(stop=-2)
    table['cb_type'] = table['cb'].str.slice(start=-2)

    table['amt_cr'] = np.where(table['amt_type']=='Cr',table['amt_value'],0)
    table['amt_dr'] = np.where(table['amt_type']=='Dr',table['amt_value'],0)
    table['cb_cr'] = np.where(table['cb_type']=='Cr',table['cb_value'],0)
    table['cb_dr'] = np.where(table['cb_type']=='Dr',table['cb_value'],0)

    table['date'] = table['date'].str.replace(" ","")

    table['txn_details'] = table['txn_details'].str.strip()

    table = table.drop(['amt_value','amt_type','cb_value','cb_type'],axis=1)

    table.columns = ['Date','Summary','Amount','Cashback','Amount Credit','Amount Debit','Cashback Credit','Cashback Debit']
    
    return table

In [20]:
def pdf_to_csv(filename):
    tables = camelot.read_pdf('./pdf/'+filename,password=cred.password, pages='all')
    
    print('extrating table 1....')
    
    temp1 = {
    'date' : tables[1].df[1][1].split("\n"),
    'txn_details' : tables[1].df[2][1].split("\n"),
    'amt' : tables[1].df[4][1].split("\n"),
    'cb' : tables[1].df[5][1].split("\n")
    }
    
    tab1 = df_creator(temp1)
    
    try:
        print('extrating table 2....')
    
        temp2 = {
        'date' : tables[5].df[0][0].split("\n"),
        'txn_details' : tables[5].df[1][0].split("\n"),
        'amt' : tables[5].df[3][0].split("\n"),
        'cb' : tables[5].df[4][0].split("\n")
        }
        tab2 = df_creator(temp2)
    
        print('concating table 1&2....')
        tab0 = pd.concat([tab1,tab2])
    except:
        tab0 = tab1
    
    print('Saving csv.....')
    tab0.to_csv('./csv/'+filename[:-4]+'.csv')
    

In [22]:
# for filename in filenames:
#     print('processing '+filename+'....')
#     pdf_to_csv(filename)
pdf_to_csv(filenames[3])

extrating table 1....
creating df.....
extrating table 2....
Saving csv.....
